<a href="https://colab.research.google.com/github/thomasp05/GIF-7005-Projet/blob/features%2Ftest-unet/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pre-configuration

## Mount Google Drive

In [1]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Clone and pull github repository

In [2]:
# Make sure clone at root
%cd /content

!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

import os
os.chdir('gif-705-projet')

# Add the local_modules directory to the set of paths
# Python uses to look for imports.
import sys
sys.path.append('/content/gif-705-projet')

/content
     |████████████████████████████████| 1.9MB 12.1MB/s 
Cloning into 'gif-705-projet'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 152 (delta 84), reused 89 (delta 42), pack-reused 0
Receiving objects: 100% (152/152), 131.34 KiB | 235.00 KiB/s, done.
Resolving deltas: 100% (84/84), done.


Checkout branch

In [3]:
!git checkout "features/test-unet" #develop
!git pull

Branch 'features/test-unet' set up to track remote branch 'features/test-unet' from 'origin'.
Switched to a new branch 'features/test-unet'
Already up to date.


# Imports and load

In [4]:
import time

import torch


from dataset import *

from models import *
import models_parts
from train_helper import *

from torch.optim import lr_scheduler


torch.manual_seed(111)

## HYPERPARAMETERS

In [6]:
N_EPOCH = 1
start_epoch = 0
BATCH_SIZE = 4
google_drive_path = "/content/drive/My Drive/GIF-7005-Projet/"
checkpoint_path = google_drive_path + "checkpoint/" 

## Load dataset

In [11]:
dataset = dcm_dataset(google_drive_path + 'GIF-7005-Projet/gif-7005-projet/data', transforms=Downsample())
print("Found {} images".format(len(dataset.img_files)))

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

dataloaders = {
  'train': train_loader,
  'val': test_loader
}

Found 26684 images


# UNet

Instantiate Model, optimizer and loss function

In [6]:
model = UNet(n_channels=1, n_classes=1).to("cuda:0")

optim = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optim, step_size=8, gamma=0.1)

## Load model checkpoint

In [ ]:
file_path = checkpoint_path + "unet_drive.pt"
model, optim, start_epoch, best_loss = load_checkpoint(model, optim, file_path)
transfer_optimizer_parts(optim)

## Simple Train

In [ ]:
# Test inference
timer = time.time()
for epoch in range(start_epoch, N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 4794.29


In [ ]:
#checkpoint_path = google_drive_path + "unet_drive.pt"
#torch.save(model.state_dict(), checkpoint_path)

save_checkpoint(epoch, model, optim, loss, file_path)

In [ ]:
checkpoint_path = google_drive_path + "out_images_after_train.pt"
torch.save(out, checkpoint_path)

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()]) #


NameError: ignored

## Train

In [9]:

#checkpoint_path = google_drive_path + "unet_train_8_epochs.pt"

model = train_model(model, optim, exp_lr_scheduler, dataloaders, N_EPOCH, file_path, best_loss)

Epoch 1/8
--------------------


KeyboardInterrupt: ignored

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
for img, (target, bounding_box) in test_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    break

plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()])


NEW TEST

In [ ]:
def visualize_model(model, num_images=4):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, (_, labels)) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


# ResNet UNet

Instantiate Model

In [1]:
model = ResNetUNet(n_class=1).to(device)

optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optim, step_size=8, gamma=0.1)

NameError: ignored

Freeze backbone layers

In [ ]:
for l in model.base_layers:
  for param in l.parameters():
    param.requires_grad = False

## Load model checkpoint

In [ ]:
file_path = checkpoint_path + "resnet_unet.pt"
model, optim, start_epoch, best_loss = load_checkpoint(model, optim, file_path)
transfer_optimizer_parts(optim)

## Simple Train

In [ ]:
# Test inference

timer = time.time()
for epoch in range(N_EPOCH):
  for img, (target, bounding_box) in train_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    loss = criterion(out, bounding_box)
    loss.backward()

    optim.step()

  print("Epoch : {}".format(epoch+1))
  print("Time  : {:.2f}".format(time.time()-timer))

Epoch : 1
Time  : 4794.29


In [ ]:
#checkpoint_path = google_drive_path + "ResNetUNet_drive.pt"
#torch.save(model.state_dict(), checkpoint_path)

save_checkpoint(epoch, model, optim, loss, file_path)

In [ ]:
checkpoint_path = google_drive_path + "out_images_after_train_ResNetUNet.pt"
torch.save(out, checkpoint_path)

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()]) #


NameError: ignored

## Train

In [ ]:
model = train_model(model, optim, exp_lr_scheduler, dataloaders, N_EPOCH, file_path, best_loss)

Epoch 1/8
--------------------


KeyboardInterrupt: ignored

### Print images

Left: Input image, Middle: Correct mask, Rigth: Predicted mask

In [ ]:
for img, (target, bounding_box) in test_loader:
    
    optim.zero_grad()

    img = img.to("cuda:0")
    bounding_box = bounding_box.to("cuda:0")

    out = model(img)

    break

plot_side_by_side([img.cpu().squeeze(), bounding_box.cpu().squeeze().numpy(), out.detach().cpu().squeeze()])
